In [1]:
import numpy as np

from sklearn import datasets
from sklearn import ensemble

from evidently.options import ColorOptions
from evidently.test_suite import TestSuite
from evidently.tests import *

In [11]:
from IPython.display import HTML

In [4]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame=True)
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan


/home/devuser/workspace/backend/.venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
#Dataset for Regression
housing_data = datasets.fetch_california_housing(as_frame=True)
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [7]:
#Dataset for binary label and probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame=True)
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

bcancer_label_ref = bcancer_ref.copy(deep=True)
bcancer_label_cur = bcancer_cur.copy(deep=True)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

bcancer_label_ref['prediction'] = model.predict(bcancer_label_ref[bcancer_data.feature_names.tolist()])
bcancer_label_cur['prediction'] = model.predict(bcancer_label_cur[bcancer_data.feature_names.tolist()])

In [9]:
#Dataset for Multiclass Classifcation (labels)
iris_data = datasets.load_iris(as_frame=True)
iris = iris_data.frame

iris_ref = iris.sample(n=75, replace=False)
iris_cur = iris.sample(n=75, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=3)
model.fit(iris_ref[iris_data.feature_names], iris_ref.target)

iris_ref['prediction'] = model.predict(iris_ref[iris_data.feature_names])
iris_cur['prediction'] = model.predict(iris_cur[iris_data.feature_names])


# Run the TestCase

In [10]:
#dataset-level tests
prob_classification_performance_dataset_tests = TestSuite(tests=[
    TestAccuracyScore(),
    TestPrecisionScore(),
    TestRecallScore(),
    TestF1Score(),
    TestRocAuc(),
    TestLogLoss(),
    TestPrecisionByClass(label=0),
    TestPrecisionByClass(label=1),
    TestRecallByClass(label=0),
    TestRecallByClass(label=1),
    TestF1ByClass(label=0),
    TestF1ByClass(label=1),

])

prob_classification_performance_dataset_tests.run(reference_data=None, current_data=bcancer_cur)
HTML(prob_classification_performance_dataset_tests.get_html())